In [1]:
import os
import pandas as pd
import numpy as np

In [2]:

# Declare available methods in local_binary_pattern
# from https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.local_binary_pattern
METHODS = ['default', 'ror','uniform','nri_uniform','var']
METHOD_INDEX = 0

# Declare the path to data feature
DATA_LBP_PATH = 'dataset/fish_data_lbp/resize/'
DATA_HOG_PATH = 'dataset/fish_data_hog'
# DATA_HL_PATH = 'dataset/fish_data_houghline'



In [3]:
# import the data feature into pandas

# uncomment line below if using lbp
df = pd.read_csv(os.path.join(DATA_LBP_PATH,METHODS[METHOD_INDEX],METHODS[METHOD_INDEX]+'_dense.csv'), header=None)

# uncomment line below if using hog
# df = pd.read_csv(os.path.join(DATA_HOG_PATH,'hog_fish.csv'), header=None)

# for some reason there are typo for the class gymnocranius_ microdon
# it needs to be removed so there are no duplicate data
df = df.drop(df[df[1]=='gymnocranius_ microdon'].index)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,250,251,252,253,254,255,256,257,258,259
0,1,A73EGS-P,1,0.029996,0.051033,0.004477,0.001441,0.004401,0.037168,0.002105,...,0.000191,0.004796,0.009656,0.009222,0.000867,0.033776,0.005344,0.004043,0.003010,0.276747
1,2,A73EGS-P,2,0.032996,0.054222,0.006582,0.000778,0.002003,0.015242,0.002207,...,0.000408,0.005791,0.001633,0.002640,0.000931,0.022054,0.002513,0.005867,0.003992,0.522372
2,3,A73EGS-P,3,0.030999,0.032615,0.003571,0.001173,0.002704,0.033903,0.002092,...,0.000102,0.005651,0.007015,0.006849,0.000676,0.032474,0.004209,0.005332,0.001901,0.317360
3,4,A73EGS-P,4,0.032999,0.060166,0.008686,0.001952,0.004936,0.032615,0.002232,...,0.000230,0.011824,0.008865,0.012985,0.000931,0.029158,0.008355,0.010867,0.005000,0.154145
4,5,A73EGS-P,5,0.034002,0.037921,0.003508,0.001008,0.003367,0.035204,0.001441,...,0.000128,0.003686,0.010599,0.010906,0.000332,0.029770,0.005357,0.003801,0.002513,0.300842


In [7]:
dfg = df.groupby(df[1])[0].count()
dfg.head()

1
A73EGS-P                    7
CUNWCB-Y                    9
Istiophorus_platypterus     6
P1ROZC-Z                   17
PQV7DP-S                    7
Name: 0, dtype: int64

In [ ]:
# set the index column for the class and print the length
COLUMN_CLASS = 1
len(df[COLUMN_CLASS].unique())

In [ ]:
# Processing One hot encoded for the class
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(df[COLUMN_CLASS].unique())
new_class = lb.transform(df[COLUMN_CLASS])
new_class.shape

In [ ]:
# set list of column of features
index_input =[i for i in range(4,len(df.columns))]

In [ ]:
# import the MLP and train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# split the data into test and train data
X_train, X_test, y_train, y_test = train_test_split(df[index_input].copy().to_numpy(), new_class, 
                                                    test_size=0.3,random_state=1,stratify=new_class)

In [ ]:
# Make MLP model and fit (train) it
clf = MLPClassifier(hidden_layer_sizes=(400,500,400,350,250),solver='adam',random_state=1,n_iter_no_change=100, max_iter=1000, learning_rate_init=0.001).fit(X_train, y_train)

In [ ]:
# show the loss training in line diagram
import matplotlib.pyplot as plt

plt.plot(clf.loss_curve_, 'r')
plt.show()

In [ ]:
# save the model
import pickle
filename = 'model/lbp_11.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
# test accuracy of the model using test data
clf.score(X_test, y_test)